# Estimate Neural Manifold Curvature

### Set-up + Imports

In [1]:
import setup

setup.main()
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

import os

import matplotlib.pyplot as plt
import numpy as np
import skdim

import neurometry.datasets.synthetic as synthetic
from neurometry.dimension.dimension import (
    plot_dimension_experiments,
    skdim_dimension_estimation,
)

os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs

Working directory:  /home/facosta/neurometry/neurometry
Directory added to path:  /home/facosta/neurometry
Directory added to path:  /home/facosta/neurometry/neurometry


In [2]:
from neurometry.curvature.main import (
    create_model_and_train_test,
    training_plot_log,
    curvature_compute_plot_log,
)

/home/facosta/neurometry/neurometry/results/configs
/home/facosta/neurometry/neurometry/results/trained_models


In [4]:
import neurometry.curvature.default_config as default_config

sweep_config = {
    "lr": 0.01,
    "batch_size": 64,
    "encoder_width": 400,
    "encoder_depth": 8,
    "decoder_width": 200,
    "decoder_depth": 4,
    "drop_out_p": 0.1,
    "wandb": {
        "project": "neurometry",
        "api_key": default_config.api_key,
    },
}

In [5]:
dataset_name = "s1_synthetic"
sweep_name = f"{dataset_name}"
fixed_config = {
    # Parameters constant across runs of the sweep (unique value):
    "dataset_name": dataset_name,
    "sweep_name": sweep_name,
    "expt_id": None,
    "timestep_microsec": None,
    "smooth": None,
    "select_gain_1": None,
    "n_times": 2500,
    "embedding_dim": 5,
    "geodesic_distortion_amp": 0.4,
    "noise_var": 0.1,
    "grid_scale": None,
    "arena_dims": None,
    "n_cells": None,
    "grid_orientation_mean": None,
    "grid_orientation_std": None,
    "field_width": None,
    "resolution": None,
    # Parameters fixed across runs and sweeps
    # (unique value depending on dataset_name):
    "manifold_dim": default_config.manifold_dim[dataset_name],
    "latent_dim": default_config.latent_dim[dataset_name],
    "posterior_type": default_config.posterior_type[dataset_name],
    "geodesic_distortion_func": default_config.geodesic_distortion_func[dataset_name],
    "n_wiggles": default_config.n_wiggles[dataset_name],
    "radius": default_config.radius[dataset_name],
    "major_radius": default_config.major_radius[dataset_name],
    "minor_radius": default_config.minor_radius[dataset_name],
    "synthetic_rotation": default_config.synthetic_rotation[dataset_name],
    # Else:
    "device": default_config.device,
    "log_interval": default_config.log_interval,
    "checkpt_interval": default_config.checkpt_interval,
    "batch_shuffle": default_config.batch_shuffle,
    "scheduler": default_config.scheduler,
    "n_epochs": default_config.n_epochs,
    "alpha": default_config.alpha,
    "beta": default_config.beta,
    "gamma": default_config.gamma,
    "gamma_moving": default_config.gamma_moving,
    "gamma_dynamic": default_config.gamma_dynamic,
    "sftbeta": default_config.sftbeta,
    "gen_likelihood_type": default_config.gen_likelihood_type,
    "n_grid_points": default_config.n_grid_points,
}

In [6]:
import wandb
import json
import logging
import neurometry.curvature.datasets.utils as utils

wandb.init(config=sweep_config)
wandb_config = wandb.config
wandb_config.update(fixed_config)
# wandb_config.update(sweep_config)

run_name = "run_" + wandb.run.id + "_" + sweep_name
wandb.run.name = run_name

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/facosta/.netrc


In [8]:
# Load data, labels
dataset, labels, train_loader, test_loader = utils.load(wandb_config)
data_n_times, data_dim = dataset.shape
wandb_config.update(
    {
        "run_name": run_name,
        "results_prefix": run_name,
        "data_n_times": data_n_times,
        "data_dim": data_dim,
    }
)

# Save config for easy access from notebooks
wandb_config_path = os.path.join(default_config.configs_dir, run_name + ".json")
with open(wandb_config_path, "w") as config_file:
    json.dump(dict(wandb_config), config_file)

# Note: loaders put data on GPU during each epoch
dataset = dataset.to(wandb_config.device)
train_losses, test_losses, model = create_model_and_train_test(
    wandb_config, train_loader, test_loader
)
logging.info(f"Done: training for {run_name}")

training_plot_log(wandb_config, dataset, labels, train_losses, test_losses, model)
logging.info(f"Done: training's plot & log for {run_name}")

curvature_compute_plot_log(wandb_config, dataset, labels, model)
logging.info(f"Done: curvature's compute, plot & log for {run_name}")
logging.info(f"\n------> COMPLETED run: {run_name}\n")

# Wandb records a run as finished even if it has failed.
wandb.finish()

Dataset shape: torch.Size([2500, 5]).
Train Epoch: 1 [0/1750 (0%)]	Loss: 0.152046
Train Epoch: 1 [1280/1750 (71%)]	Loss: 0.139183
====> Epoch: 1 Average loss: 76.3644
====> Test set loss: 0.2465
Train Epoch: 2 [0/1750 (0%)]	Loss: 0.136094
Train Epoch: 2 [1280/1750 (71%)]	Loss: 0.137244
====> Epoch: 2 Average loss: 0.1681
====> Test set loss: 0.2463
Train Epoch: 3 [0/1750 (0%)]	Loss: 0.135392
Train Epoch: 3 [1280/1750 (71%)]	Loss: 0.137922
====> Epoch: 3 Average loss: 0.1678
====> Test set loss: 0.2453
Train Epoch: 4 [0/1750 (0%)]	Loss: 0.134395
Train Epoch: 4 [1280/1750 (71%)]	Loss: 0.135546
====> Epoch: 4 Average loss: 0.1677
====> Test set loss: 0.2447
Train Epoch: 5 [0/1750 (0%)]	Loss: 0.132604
Train Epoch: 5 [1280/1750 (71%)]	Loss: 0.138701
====> Epoch: 5 Average loss: 0.1670
====> Test set loss: 0.2444
Train Epoch: 6 [0/1750 (0%)]	Loss: 0.129852
Train Epoch: 6 [1280/1750 (71%)]	Loss: 0.140392
====> Epoch: 6 Average loss: 0.1669
====> Test set loss: 0.2464
Train Epoch: 7 [0/1750 (0

INFO:root:Done: training for run_zq9ohvqh_s1_synthetic


====> Test set loss: 0.1779


INFO:root:Done: training's plot & log for run_zq9ohvqh_s1_synthetic


Computing learned curvature...


TypeError: PullbackMetric.__init__() got an unexpected keyword argument 'dim'